In [ ]:
'''
face/eyes detection
抓脸
'''

import numpy as np
import cv2 as cv
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
cap = cv.VideoCapture(0)
num = 0

while(True):
    # 一帧一帧捕捉
    ret, frame = cap.read()
    # 我们对帧的操作在这里
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        eyes_found = False
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,2)
        for (ex,ey,ew,eh) in eyes:
            if not eyes_found: 
                eyes_found = True
                face_img = frame[y-10:y+h+10, x-10:x+w+10]
                image_name = './ning/%d.jpg' % (num)
                cv.imwrite(image_name,face_img)
                num += 1
                cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    cv.imshow('img',frame)
    if cv.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
'''
训练
'''
import os
import numpy as np
import cv2

#脸部检测函数
def face_detect_demo(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = face_detector.detectMultiScale(gray, 1.2, 6)
    # 如果未检测到面部，则返回原始图像
    if (len(faces) == 0):
        return None, None
    # 目前假设只有一张脸，xy为左上角坐标，wh为矩形的宽高
    (x, y, w, h) = faces[0]
    # 返回图像的脸部部分
    return gray[y:y + w, x:x + h], faces[0]

def ReFileName(dirPath):
    """
    :param dirPath: 文件夹路径
    :return:
    """
    # 对目录下的文件进行遍历
    faces=[]
    for file in os.listdir(dirPath):
        # 判断是否是文件
        if os.path.isfile(os.path.join(dirPath, file)) == True:
           c= os.path.basename(file)
           name = dirPath + '\\' + c
           img = cv2.imread(name)
           # 检测脸部
           face, rect = face_detect_demo(img)
           # 我们忽略未检测到的脸部
           if face is not None:
               # 将脸添加到脸部列表并添加相应的标签
               faces.append(face)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces

#David
dirDavid = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\david"#文件路径
Davidfaces=ReFileName(dirDavid)#调用函数
labelDavidfaces=np.array([0 for i in range(len(Davidfaces))])#标签处理

#Ning
dirNing = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\ning"#文件路径
Ningfaces=ReFileName(dirNing)#调用函数
labelNingfaces=np.array([1 for i in range(len(Ningfaces))])#标签处理

#Jamesfaces
dirMyfaces = r"C:\james_home\local_src\codebase\pyexec\opencv\face_recognition\james"#文件路径
Jamesfaces=ReFileName(dirMyfaces)#调用函数
labelJamesfaces=np.array([2 for i in range(len(Jamesfaces))])#标签处理

#拼接并打乱数据特征和标签
x=np.concatenate((Jamesfaces,Davidfaces,Ningfaces),axis=0)
y=np.concatenate((labelJamesfaces,labelDavidfaces,labelNingfaces),axis=0)

index = [i for i in range(len(y))] # test_data为测试数据
np.random.seed(1)
np.random.shuffle(index) # 打乱索引
train_data = x[index]
train_label = y[index]

#分类器
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(train_data, train_label)
# 保存训练数据
recognizer.write('train.yml')

In [ ]:
'''
预测
'''
import cv2
#人脸检测函数
def face_detect_demo(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
    faces = face_detector.detectMultiScale(gray, 1.2, 6)
    # 如果未检测到面部，则返回原始图像
    if (len(faces) == 0):
        return None, None
    # 目前假设只有一张脸，xy为左上角坐标，wh为矩形的宽高
    (x, y, w, h) = faces[0]
    roi_gray = gray[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray,1.1,2)
    if (len(eyes) == 0):
        return None, None 
    # 返回图像的脸部部分
    return gray[y:y + w, x:x + h], faces[0]

# 根据给定的人脸（x，y）坐标和宽度高度在图像上绘制矩形
def draw_rectangle(img, rect):
    (x, y, w, h) = rect#矩形框
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)

# 根据给定的人脸（x，y）坐标写出人名
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (128, 128, 0), 2)

# 此函数识别图像中的人物并在脸部周围绘制一个矩形及其人名
facelabel = ["David", "Ning", "James"]#人物名

#导入训练结果
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('train.yml')#读取前文训练的结果

def predict(image):
 
    # 检测人脸区域
    face, rect = face_detect_demo(image)#face_detect_demo前面的人脸检测函数
    #print(rect)=[x,y,w,h]
    # 预测人脸名字
    if not face is None:
        label = recognizer.predict(face)
        print(label)#label[0]为名字，label[1]可信度数值越低，可信度越高（
        if label[1]<=50:
            # 获取由人脸识别器返回的相应标签的人名
            label_text = facelabel[label[0]]

            # 在检测到的脸部周围画一个矩形
            draw_rectangle(image, rect)
            # 标出预测的人名
            draw_text(image, label_text, rect[0], rect[1])
            # 返回预测的图像
            return image
        else:
            # 在检测到的脸部周围画一个矩形
            draw_rectangle(image, rect)
            # 标出预测的人名
            draw_text(image, "not find", rect[0], rect[1])
            # 返回预测的图像
            return image
    else:
        return image

cap = cv2.VideoCapture(0)
while(True):
    # 一帧一帧捕捉
    ret, frame = cap.read()
    # 我们对帧的操作在这里
    pred_img = predict(frame)
    cv2.imshow('img',pred_img)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()